In [1]:
import cv2 
import numpy as np
from skimage import data, io, color, transform, exposure
import os
from matplotlib import pyplot as plt
from threading import Thread
from queue import Queue
import json
from pprint import pprint

class FileVideoStream:
    def __init__(self, path, queueSize = 512):
        self.stream = cv2.VideoCapture(path)
        self.stopped = False
        self.Q = Queue(maxsize = queueSize)
        
    def start(self):
        t = Thread(target = self.update, args = ())
        t.daemon = True
        t.start()
        return self
    
    def update(self):
        while True:
            if self.stopped:
                return
            
            if not self.Q.full():
                (grabbed, frame) = self.stream.read()
                if not grabbed:
                    self.stop()
                    return
                
                self.Q.put(frame)
    def read(self):
        return self.Q.get()
    
    def more(self):
        return self.Q.qsize() > 0
    
    def stop(self):
        self.stopped = True


In [2]:
def findmaximumpeople(json_folder_dir, jsons, id0, idn):
    
    if not id0 == idn:
        midval = (id0 + idn)//2
        leftval = findmaximumpeople(json_folder_dir, jsons, id0, midval)
        rightval = findmaximumpeople(json_folder_dir, jsons, midval + 1, idn)
        
        if leftval >= rightval:
            return leftval
        else:
            return rightval
    else:
        with open((json_folder_dir + jsons[id0])) as f:
            data = json.load(f)
        
        total_number_of_people = len(data['people'])
        
#         for i in range(0, total_number_of_people - 1):
#             total_points = len(data['people'][i]['pose_keypoints_2d'])
#             division = total_points // 3
#             summed_certainty = 0
#             for j in range(0, total_points + 1, 3):
# #                 print(data['people'][i]['pose_keypoints_2d'][j - 1])
#                 if j != 0:
#                     summed_certainty += data['people'][i]['pose_keypoints_2d'][j - 1]

            
#             average_certainty = summed_certainty / division
            
#             if average_certainty < 0.70:
#                 total_number_of_people -= 1
                
        return(total_number_of_people)
    

In [3]:
def data_points(jsons, json_folder_dir):
    data_points = []
    
    for i in range(0, len(jsons)):
        data_points.append([])
        with open((json_folder_dir + jsons[i])) as f:
            data = json.load(f)
        
        n_people = len(data['people'])
    
        for j in range(0, n_people):
            data_points[i].append([])
            datas = data['people'][j]['pose_keypoints_2d']
        
            for k in range(0, len(datas)):
                data_points[i][j].append(datas[k])
                
    return data_points

In [4]:
json_folder = 'DS3'
json_folder_slash = 'DS3/'

jsons = [json for json in os.listdir(json_folder) if json.endswith(".json")]

# with open((json_folder_slash + jsons[60])) as f:
#     data = json.load(f)

number_of_people = findmaximumpeople(json_folder_slash, jsons, 0, (len(jsons) - 1))

data_points1 = data_points(jsons, json_folder_slash)



In [5]:
print(data_points1[0])
print(len(data_points1[300]))

[]
2


In [6]:
i = 0

loading = False

# video = cv2.VideoWriter('data points 2.avi',cv2.VideoWriter_fourcc(*"MJPG"), 30, (1920,1080),True)

fvs = FileVideoStream("result2.mp4").start()

while not fvs.more():
    if not loading:
        print("Loading")
        loading = True

while fvs.more():
    frame = fvs.read()
    people = len(data_points1[i])
    if people != 0:
        for j in range(0, people):
            points = data_points1[i][j]
            n_points = len(points)
            div = n_points // 3
            for k in range(0, div):
                cv2.putText(frame, str([points[0 + 3 * k], points[1 + 3 * k]]),(int(points[0 + 3 * k]) - 10,int(points[1 + 3 * k])), cv2.FONT_HERSHEY_SIMPLEX, 0.3,(255,255,255),1,cv2.LINE_AA)
                
#     video.write(frame)
    cv2.imshow("Video", frame)
    i += 1
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break
        
fvs.stop()
cv2.destroyAllWindows()
# video.release()

Loading
